In [1]:
from btcinfocharts_scraper import grab_the_data

In [5]:
df, day = grab_the_data()

Not all the data is available for today, 2022-04-17.
Instead yesterday's 2022-04-16 data will be used.


In [18]:
from datetime import date, timedelta
import re
import pandas as pd
from bs4 import BeautifulSoup #module for web scraping install by pip install beautifulsoup4
import requests #for requesting html. install by pip install requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import re #regular expression for data extraction by pattern matching. installed by default.

In [11]:

# get the current date
today_var1 = date.today()
today = str(today_var1).replace('-', '/')


# get the current date
today_var2 = date.today()
#subtract one day
yestday = today_var2 - timedelta(days=1)

yesterday = str(yestday).replace('-', '/')
  
print(yesterday, today)

2022/04/16 2022/04/17


In [15]:
# the list of features to scrape
features = [
                'median_transaction_fee3momUSD',
                'fee_to_reward7momUSD',
                'top100cap7mom',
                'mining_profitability7rsi',
                'top100cap14mom',
                'price3wmaUSD',
                'transactionvalue90emaUSD',
                'difficulty30sma',
                'fee_to_reward90smaUSD'
                ]

url_list = []

for feat in features:
    regex = re.compile(r'(top100cap|[a-z_]+)(\d+\w+)')
    re_result = regex.search(feat)

    feat = re_result.group(1)
    trans = re_result.group(2)

    url='https://bitinfocharts.com/comparison/'+feat+'-'+'btc'+'-'+trans+'.html'
    url_list.append(url)


print(url_list)
print('\n', features)

['https://bitinfocharts.com/comparison/median_transaction_fee-btc-3momUSD.html', 'https://bitinfocharts.com/comparison/fee_to_reward-btc-7momUSD.html', 'https://bitinfocharts.com/comparison/top100cap-btc-7mom.html', 'https://bitinfocharts.com/comparison/mining_profitability-btc-7rsi.html', 'https://bitinfocharts.com/comparison/top100cap-btc-14mom.html', 'https://bitinfocharts.com/comparison/price-btc-3wmaUSD.html', 'https://bitinfocharts.com/comparison/transactionvalue-btc-90emaUSD.html', 'https://bitinfocharts.com/comparison/difficulty-btc-30sma.html', 'https://bitinfocharts.com/comparison/fee_to_reward-btc-90smaUSD.html']

 ['median_transaction_fee3momUSD', 'fee_to_reward7momUSD', 'top100cap7mom', 'mining_profitability7rsi', 'top100cap14mom', 'price3wmaUSD', 'transactionvalue90emaUSD', 'difficulty30sma', 'fee_to_reward90smaUSD']


In [16]:
df_features=pd.DataFrame(features,columns=['Features']) # convert feature list to dataframe
df_url=pd.DataFrame(url_list,columns=['URL']) # convert url list to dataframe

assert len(df_features) == len(df_url)

print(df_url) 
print('\n', df_features)

                                                 URL
0  https://bitinfocharts.com/comparison/median_tr...
1  https://bitinfocharts.com/comparison/fee_to_re...
2  https://bitinfocharts.com/comparison/top100cap...
3  https://bitinfocharts.com/comparison/mining_pr...
4  https://bitinfocharts.com/comparison/top100cap...
5  https://bitinfocharts.com/comparison/price-btc...
6  https://bitinfocharts.com/comparison/transacti...
7  https://bitinfocharts.com/comparison/difficult...
8  https://bitinfocharts.com/comparison/fee_to_re...

                         Features
0  median_transaction_fee3momUSD
1           fee_to_reward7momUSD
2                  top100cap7mom
3       mining_profitability7rsi
4                 top100cap14mom
5                   price3wmaUSD
6       transactionvalue90emaUSD
7                difficulty30sma
8          fee_to_reward90smaUSD


In [22]:
url_list = ['https://bitinfocharts.com/comparison/median_transaction_fee-btc-3momUSD.html',
            'https://bitinfocharts.com/comparison/fee_to_reward-btc-7momUSD.html',
            'https://bitinfocharts.com/comparison/top100cap-btc-7mom.html',
            'https://bitinfocharts.com/comparison/mining_profitability-btc-7rsi.html',
            'https://bitinfocharts.com/comparison/top100cap-btc-14mom.html',
            'https://bitinfocharts.com/comparison/price-btc-3wmaUSD.html',
            'https://bitinfocharts.com/comparison/transactionvalue-btc-90emaUSD.html',
            'https://bitinfocharts.com/comparison/difficulty-btc-30sma.html',
            'https://bitinfocharts.com/comparison/fee_to_reward-btc-90smaUSD.html']

In [25]:
for x in range(len(url_list)):
    
    url = url_list[x]
    feature = features[x]

    session = requests.Session()
    retry = Retry(connect=10, backoff_factor=3)
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    page=session.get(url)

    #create the beautiful soup object
    soup = BeautifulSoup(page.content, 'html.parser')
    # extract the portion with dates and values
    all_soup = soup.find_all('script')[4]
    # convert the data into string format
    soup_string = str(all_soup.string)

    # assign current date
    tday = today # create this variable in the code

    # create a regular expression for extracting the value associated with the date
    regex = tday + '\"\),([0-9\.]+)'
    soup_regex = re.compile(regex)
    # call that regular expression on the html string
    regex_result = soup_regex.search(soup_string)

    # create a conditional statement for when current date's data has not been posted
    if regex_result is None:
        today_feat_val = None
    else:
        today_feat_val = regex_result.group(1)

    # extract values for yesterday
    yday = yesterday # create this variable in the code
    regex = yday + '\"\),([0-9\.]+)'
    soup_regex = re.compile(regex)
    regex_result = soup_regex.search(soup_string)

    # isolate just the feature value for the current date
    yesterday_feat_val = regex_result.group(1)

    # create a dictionary for each day and the values
    feat_val_dic = {
                    tday:
                        {feature:today_feat_val},
                    yday:
                        {feature:yesterday_feat_val}
                   }


    if x == 0:
        df = pd.DataFrame.from_dict(feat_val_dic)
    else:
        df_2 = pd.DataFrame.from_dict(feat_val_dic)
        df = pd.concat([df, df_2])
        
        
df

,2022/04/17,2022/04/16
median_transaction_fee3momUSD,None,0.332
fee_to_reward7momUSD,None,0.762
top100cap7mom,15.423,15.423
mining_profitability7rsi,None,0.174
top100cap14mom,15.423,15.423
price3wmaUSD,40013,40399
transactionvalue90emaUSD,None,571615
difficulty30sma,None,28225928151211
fee_to_reward90smaUSD,None,0.762


In [26]:
# create a list of values for today to check for None type
today_val_list = [val for val in df.iloc[:, 0]]

# get the current date
tday = date.today()
#subtract one day
yday = tday - timedelta(days=1)

In [28]:
# check if None is in today's date
if None in today_val_list:
    print('Not all the data is available for today, {}.'.format(tday))
    print('Instead yesterday\'s {} data will be used.'.format(yday))
    # create a dataframe from yesterday's data to be returned
    yesterday_df = pd.DataFrame(df.iloc[:,1])
    yesterday_df = yesterday_df.T
    print(yesterday_df, yday.strftime('%Y/%m/%d'))

else:
    print('All of the data for today {} is available.'.format(tday))
    print('Today\'s {} data will be used'.format(tday))
    # create a dataframe from today's data to be returned
    today_df = pd.DataFrame(df.iloc[:, 0])
    today_df = today_df.T
    print(today_df, tday.strftime('%Y/%m/%d'))

Not all the data is available for today, 2022-04-17.
Instead yesterday's 2022-04-16 data will be used.
           median_transaction_fee3momUSD fee_to_reward7momUSD top100cap7mom  \
2022/04/16                         0.332                0.762        15.423   

           mining_profitability7rsi top100cap14mom price3wmaUSD  \
2022/04/16                    0.174         15.423        40399   

           transactionvalue90emaUSD difficulty30sma fee_to_reward90smaUSD  
2022/04/16                   571615  28225928151211                 0.762   2022/04/16
